Copyright **`(c)`** 2021 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see 'LICENCE.md' for details.

# Connect 4

In [26]:
from collections import Counter
import numpy as np
import math
import random


In [49]:
COLS = 7
ROWS = 6
FOUR = 4
PLAYER_PIECE = 1
AI_PIECE = 2

# Board can be initiatilized with `board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)`
# Notez Bien: Connect 4 "columns" are actually NumPy "rows"

## Basic Functions

In [67]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(ROWS) if board[n,COLS  - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    #(index,) = next((i for i, v in np.ndenumerate(board[0][column]) if v == 0))
    index = next((i for i in range(ROWS-1, -1, -1) if board[i][column] == 0 ))
    board[index,column] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(ROWS)
            for r in (list(range(n, n + FOUR)) for n in range(COLS - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLS)
            for c in (list(range(n, n + FOUR)) for n in range(ROWS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                ( range(co, co + FOUR),range(ro, ro + FOUR))
                for ro in range(0, COLS - FOUR + 1)
                for co in range(0, ROWS - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(co + FOUR - 1, co - 1, -1), range(ro, ro + FOUR))
                for ro in range(0, COLS - FOUR + 1)
                for co in range(0, ROWS - FOUR + 1)
            )
        )
    )

def is_terminal_node(board):
    return four_in_a_row(board, PLAYER_PIECE) or four_in_a_row(board, AI_PIECE) or len(valid_moves(board)) == 0


## Alpha-beta pruning

In [68]:

# The algorithm calculating the best move to make given a depth of the search tree.
# Depth is how many layers algorithm scores boards. Complexity grows exponentially.
# Alpha and beta are best scores a side can achieve assuming the opponent makes the best play.
# More on alpha-beta pruning here: https://www.youtube.com/watch?v=l-hh51ncgDI.
# maximizing_palyer is a boolean value that tells whether we are maximizing or minimizing
# in this implementation, AI is maximizing.
def minimax(board, depth, alpha, beta, maximizing_player):

    # all valid locations on the board
    valid_locations = valid_moves(board)

    # boolean that tells if the current board is terminal
    is_terminal = is_terminal_node(board)

    # if the board is terminal or depth == 0
    # we score the win very high and a draw as 0
    if depth == 0 or is_terminal:
        if is_terminal: # winning move 
            if four_in_a_row(board, AI_PIECE):
                return (None, 10000000)
            elif four_in_a_row(board, PLAYER_PIECE):
                return (None, -10000000)
            else:
                return (None, 0)
        # if depth is zero, we simply score the current board
        else: # depth is zero
            return (None, score_position(board, AI_PIECE))

    # if the current board is not rerminal and we are maximizing
    if maximizing_player:

        # initial value is what we do not want - negative infinity
        value = -math.inf

        # this will be the optimal column. Initially it is random
        column = random.choice(valid_locations)

        # for every valid column, we simulate dropping a piece with the help of a board copy
        # and run the minimax on it with decresed depth and switched player
        for col in valid_locations:
            #row = get_next_open_row(board, col)
            b_copy = board.copy()
            play(b_copy, col, AI_PIECE)
            # recursive call
            new_score = minimax(b_copy, depth-1, alpha, beta, False)[1]
            # if the score for this column is better than what we already have
            if new_score > value:
                value = new_score
                column = col
            # alpha is the best option we have overall
            alpha = max(value, alpha) 
            # if alpha (our current move) is greater (better) than beta (opponent's best move), then 
            # the oponent will never take it and we can prune this branch
            if alpha >= beta:
                break

        return column, value
    
    # same as above, but for the minimizing player
    else: # for thte minimizing player
        value = math.inf
        column = random.choice(valid_locations)
        for col in valid_locations:
            #row = get_next_open_row(board, col)
            b_copy = board.copy()
            play(b_copy, col, PLAYER_PIECE)
            new_score = minimax(b_copy, depth-1, alpha, beta, True)[1]
            if new_score < value:
                value = new_score
                column = col
            beta = min(value, beta) 
            if alpha >= beta:
                break
        return column, value

def score_position(board, piece):

    score = 0

    # score center column --> we are prioritizing the central column because it provides more potential winning windows
    center_array = [int(i) for i in list(board[:,COLS//2])]
    center_count = center_array.count(piece)
    score += center_count * 6

    # below we go over every single window in different directions and adding up their values to the score
    # score horizontal
    for r in range(ROWS):
        row_array = [int(i) for i in list(board[r,:])]
        for c in range(COLS - 3):
            window = row_array[c:c + 4]
            score += evaluate_window(window, piece)

    # score vertical
    for c in range(COLS):
        col_array = [int(i) for i in list(board[:,c])]
        for r in range(ROWS-3):
            window = col_array[r:r+4]
            score += evaluate_window(window, piece)

    # score positively sloped diagonals
    for r in range(3,ROWS):
        for c in range(COLS - 3):
            window = [board[r-i][c+i] for i in range(4)]
            score += evaluate_window(window, piece)

    # score negatively sloped diagonals
    for r in range(3,ROWS):
        for c in range(3,COLS):
            window = [board[r-i][c-i] for i in range(4)]
            score += evaluate_window(window, piece)

    return score
def evaluate_window(window, piece):
    # by default the oponent is the player
    opponent_piece = PLAYER_PIECE

    # if we are checking from the player's perspective, then the oponent is AI
    if piece == PLAYER_PIECE:
        opponent_piece = AI_PIECE

    # initial score of a window is 0
    score = 0

    # based on how many friendly pieces there are in the window, we increase the score
    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(0) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(0) == 2:
        score += 2

    # or decrese it if the oponent has 3 in a row
    if window.count(opponent_piece) == 3 and window.count(0) == 1:
        score -= 4 

    return score


## Example

In [69]:
def print_board(board):
    for row in range(len(board)):
      #for space in self.board[row]:
      print(board[row]) 
      #print( )
    print( '-' * 50)

# initializing the board
board = board = np.zeros((ROWS,COLS), dtype=np.byte)

# initially nobody has won yet
game_over = False

turn = random.randint(0, 1)


# game loop
# -------------------------------
while not game_over:
    print_board(board)
    if turn == 1:
        msg = 'Which column number would you like to play? '
        col = input(msg)        
        print ('You have choosen column', col)
        play(board, int(col), PLAYER_PIECE)
        if four_in_a_row(board, PLAYER_PIECE):
            game_over = True
            print("PLAYER WINS!")
            print_board(board)
        turn += 1

    if turn == 0:
        # the column to drop in is found using minimax
        col, minimax_score = minimax(board, 5, -math.inf, math.inf, True) 
        print ('Computer chooses column', col)
        play(board, col, AI_PIECE)
        if four_in_a_row(board, AI_PIECE):
            game_over = True
            print("COMPUTER WINS!")  
            print_board(board)
        turn += 1
    turn = turn % 2

[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
--------------------------------------------------
You have choosen column 3
[0 0 0 1 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
--------------------------------------------------
Computer chooses column 3
[0 0 0 1 0 0 0]
[0 0 0 2 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
--------------------------------------------------
You have choosen column 4
[0 0 0 1 1 0 0]
[0 0 0 2 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
--------------------------------------------------
Computer chooses column 2
[0 0 2 1 1 0 0]
[0 0 0 2 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
--------------------------------------------------


KeyboardInterrupt: Interrupted by user